In [7]:

%glue_version 4.0
%additional_python_modules tgedr-nihao
# --- nn
# %region eu-central-1
# %iam_role arn:aws:iam::988864172292:role/custom_role_for_glue_interactive_sessions
# --- tgedr
%region eu-north-1
%iam_role arn:aws:iam::041651352119:role/tgedr_it_role_for_glue_service




Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 1.0.3 and you have 1.0.2 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Setting Glue version to: 4.0
Additional python modules to be included:
tgedr-nihao
Previous region: None
Setting new region to: eu-north-1
Region is set to: eu-north-1
Current iam_role is None
iam_role has been set to arn:aws:iam::041651352119:role/tgedr_it_role_for_glue_service.


In [10]:
%%configure
{
  "enable_spark_ui": "true",
  "python_version": "3.10"
}

The following configurations have been updated: {'enable_spark_ui': 'true', 'python_version': '3.10'}


In [1]:
spark

Trying to create a Glue session for the kernel.
Session Type: etl
Worker Type: G.1X
Number of Workers: 5
Session ID: 69bf9205-17ef-4b9a-addf-9abbfd90bb25
Applying the following default arguments:
--glue_kernel_version 1.0.2
--enable-glue-datacatalog true
--enable_spark_ui true
--python_version 3.10
--additional-python-modules tgedr-nihao
Waiting for session 69bf9205-17ef-4b9a-addf-9abbfd90bb25 to get into ready status...
Session 69bf9205-17ef-4b9a-addf-9abbfd90bb25 has been created.


In [2]:
import sys
import logging

from pyspark.context import SparkContext
from pyspark.sql import DataFrame

from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext
from awsglue.job import Job

from tgedr.nihao.processors.ticker_simple_analysis import TickerSimpleAnalysis

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

sys.argv+=['--source_bucket_key', "s3://de-landing-dev-c44cd1c4-19d5-4b81-af9c-37cb1abac051/dataengineering"]
sys.argv+=['--target_bucket_key', "s3://de-bronze-dev-ae67b8d7-3e23-4c66-a101-33a096537444/dataengineering"]
sys.argv+=['--job_name', 'my_test_job']

args = getResolvedOptions(
    sys.argv, ["source_bucket_key", "target_bucket_key", "job_name"]
)
#args = {"source_bucket_key": "s3://de-landing-dev-c44cd1c4-19d5-4b81-af9c-37cb1abac051/dataengineering", 
#        "target_bucket_key": "s3://de-bronze-dev-ae67b8d7-3e23-4c66-a101-33a096537444/dataengineering", 
#        "job_name": "tgedr_test"}
logger.info(f"[in] args: {args}")

In [4]:
# get glue context
glueContext = GlueContext(SparkContext.getOrCreate())

source_bucket_key = args["source_bucket_key"]
target_bucket_key = args["target_bucket_key"]
job_name = args["job_name"]

In [6]:
job = Job(glueContext)
job.init(job_name, args)

In [8]:
input = glueContext.spark_session.read.parquet(source_bucket_key)
result: DataFrame = TickerSimpleAnalysis().process(input)
    

In [9]:
result.show()

+------+-----------+------------------+---------+
|symbol|actual_time|         adj_close|   volume|
+------+-----------+------------------+---------+
|   AMD| 3.220992E8|3.1458330154418945| 219600.0|
|   AMD| 3.221856E8|           3.03125| 727200.0|
|   AMD|  3.22272E8|3.0416669845581055| 295200.0|
|   AMD| 3.223584E8|3.0104169845581055| 159600.0|
|   AMD| 3.224448E8|2.9166669845581055| 130800.0|
|   AMD|  3.22704E8|2.6666669845581055| 436800.0|
|   AMD| 3.227904E8|2.6041669845581055| 645600.0|
|   AMD| 3.228768E8|2.4479169845581055| 466800.0|
|   AMD| 3.229632E8|             2.375|1129200.0|
|   AMD| 3.230496E8|2.5416669845581055| 666000.0|
|   AMD| 3.233088E8|2.5416669845581055| 430800.0|
|   AMD| 3.233952E8|2.7395830154418945| 492000.0|
|   AMD| 3.234816E8|             2.625| 283200.0|
|   AMD|  3.23568E8|2.6354169845581055| 226800.0|
|   AMD| 3.239136E8|2.5416669845581055| 355200.0|
|   AMD|     3.24E8|           2.59375| 765600.0|
|   AMD| 3.240864E8|2.6666669845581055| 134400.0|


In [10]:
result.write.parquet(target_bucket_key)
job.commit()

In [2]:
# parquet_file = "s3://988864172292-gscdm-dev-eu-central-1-landing-zone/dh_cdm/events"
parquet_file = "s3://de-landing-dev-c44cd1c4-19d5-4b81-af9c-37cb1abac051/dataengineering"

In [3]:
df = spark.read.parquet(parquet_file)